# Comparison Between TreeValue and DM-Tree

## Run Environment Information

In [ ]:
import os
import platform
import shutil

import psutil
from hbutils.scale import size_to_bytes_str

print('OS:', platform.platform())
print('Python:', platform.python_implementation(), platform.python_version())
print('Processor:', platform.processor())
print('CPUS:', os.cpu_count())
print('Memory:', size_to_bytes_str(psutil.virtual_memory().total, precision=3))
print('Has CUDA:', 'Yes' if shutil.which('nvidia-smi') else 'No')

## Flatten and Unflatten

We create a dictionary for flatten

In [ ]:
origin = {'a': 1, 'b': 2, 'c': {'x': 3, 'y': 4}}

### TreeValue's Performance

In [ ]:
from treevalue import FastTreeValue, flatten, unflatten

origin_tree = FastTreeValue(origin)

In [ ]:
flatted = flatten(origin_tree)
flatted

In [ ]:
%timeit flatten(origin_tree)

In [ ]:
unflatten(flatted)

In [ ]:
%timeit unflatten(flatted)

### DM-Tree's Performance

This is the [dm-tree](https://github.com/deepmind/tree) library
And here is a simplest example of dm-tree.

In [ ]:
from tree import flatten, flatten_with_path

We try to flatten `origin` with both [flatten](https://tree.readthedocs.io/en/latest/api.html#tree.flatten) and [flatten_with_path](https://tree.readthedocs.io/en/latest/api.html#tree.flatten_with_path) function, and then measure their performance.

In [ ]:
flatten(origin)

In [ ]:
%timeit flatten(origin)

In [ ]:
flatten_with_path(origin)

In [ ]:
%timeit flatten_with_path(origin)

Obviously, `flatten` in dm-tree is an irreversible operation, but `flatten_with_path` is reversible, which is similar to `flatten` in treevalue.

But the [unflatten_as](https://tree.readthedocs.io/en/latest/api.html#tree.unflatten_as) is absolutely another thing in dm-tree

In [ ]:
from tree import unflatten_as

In [ ]:
unflatten_as({'a': None, 'b': None, 'c': {'x': None, 'y': None}}, [1, 2, 3, 4])

In [ ]:
%timeit unflatten_as({'a': None, 'b': None, 'c': {'x': None, 'y': None}}, [1, 2, 3, 4])

## Positional Replacement

This is the performance in treevalue

In [ ]:
from treevalue import flatten, unflatten

def replace(t, v):
    pairs = flatten(t)
    return unflatten([(path, vi) for (path, _), vi in zip(pairs, v)])

In [ ]:
replace(origin_tree, [3, 5, 7, 9])

In [ ]:
%timeit replace(origin_tree, [3, 5, 7, 9])

This is the performance in dm-tree

In [ ]:
from tree import unflatten_as

In [ ]:
unflatten_as(origin, [3, 5, 7, 9])

In [ ]:
%timeit unflatten_as(origin, [3, 5, 7, 9])

Treevalue's performance is much better.